In [1]:
import csv
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
import lda
import numpy as np
from pylab import *


corpus = []
jieba.set_dictionary('dict.txt.big')

with open('Output.csv', newline='',encoding='UTF-8') as csvfile:
    
    rows = csv.reader(csvfile)
    
    for row in rows:
        reHTTPS=re.compile(r'https://[a-zA-Z0-9.?/&=:]*',re.S)
        noHTTPS=reHTTPS.sub("",row[2])
        
        reHTTP=re.compile(r'http://[a-zA-Z0-9.?/&=:]*',re.S)
        noHTTP=reHTTP.sub("",noHTTPS)
        
        fn = noHTTP.replace(r"\u3000", "").replace(r"\n","").replace("#","")

        corpus.append(" ".join(jieba.cut(fn.split(',')[0], cut_all=False)))
        
#print(corpus)

Building prefix dict from C:\Users\Leo\Documents\dict.txt.big ...
Loading model from cache C:\Users\Leo\AppData\Local\Temp\jieba.uee93d16b7146edfccd4cb1025459943e.cache
Loading model cost 1.246 seconds.
Prefix dict has been built succesfully.


In [2]:
def get_lda_input(chapters):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X.toarray(), vectorizer

def lda_train(weight, vectorizer):
    model = lda.LDA(n_topics=5, n_iter=500, random_state=1)
    model.fit(weight)

    doc_num = len(weight)
    topic_word = model.topic_word_
    vocab = vectorizer.get_feature_names()
    titles = ["第{}章".format(i) for i in range(1, doc_num + 1)]

    n_top_words = 10000
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1]
        #print('Topic {}: {}'.format(i, ' '.join(topic_words)))
        with open('Topic{}.txt'.format(i),'a',encoding='UTF-8') as Topics:
            Topics.write(' '.join(topic_words))

In [3]:
weight, vectorizer = get_lda_input(corpus)
lda_train(weight, vectorizer)

INFO:lda:n_documents: 103
INFO:lda:vocab_size: 1116
INFO:lda:n_words: 2221
INFO:lda:n_topics: 5
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -23387
INFO:lda:<10> log likelihood: -18456
INFO:lda:<20> log likelihood: -17939
INFO:lda:<30> log likelihood: -17857
INFO:lda:<40> log likelihood: -17544
INFO:lda:<50> log likelihood: -17346
INFO:lda:<60> log likelihood: -17236
INFO:lda:<70> log likelihood: -17197
INFO:lda:<80> log likelihood: -17152
INFO:lda:<90> log likelihood: -17182
INFO:lda:<100> log likelihood: -17168
INFO:lda:<110> log likelihood: -17202
INFO:lda:<120> log likelihood: -17092
INFO:lda:<130> log likelihood: -17150
INFO:lda:<140> log likelihood: -17140
INFO:lda:<150> log likelihood: -17018
INFO:lda:<160> log likelihood: -17078
INFO:lda:<170> log likelihood: -17067
INFO:lda:<180> log likelihood: -16997
INFO:lda:<190> log likelihood: -17005
INFO:lda:<200> log likelihood: -16977
INFO:lda:<210> log likelihood: -16943
INFO:lda:<220> log likelihood: -16984
INFO:lda:<230> log l